In [1]:
#main
import numpy as np
import pandas as pd
import pandas_profiling
import json
import time
import datetime

# support
import corelib

# csv to pd loader
from corelib.data_loader import loader

# reduce memory usage for pd-index
from corelib.data_prep import reduce_mem_usage
from corelib.data_prep import reduce_obj_mem_usage

# searching for unical ordered values
from corelib.data_prep import search_func

#dump data with
from corelib.data_dump import dumper

In [39]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
pd.set_option('display.max_colwidth', 100)

С какими задачами можно столкнуться и зачем вообще питонисту работать с данными?

In [2]:
data = loader(mode='extract')
data.keys()

dict_keys(['train'])

In [3]:
df_train = data['train']
del data

- Почему данные нужно приводить к какому-то виду?
- Что за проблема вычислительной сложности?
- pandas и numpy

In [41]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 11 columns):
installation_id    200000 non-null object
event_id           200000 non-null object
game_session       200000 non-null object
timestamp          200000 non-null object
event_data         200000 non-null object
event_count        200000 non-null int64
event_code         200000 non-null int64
game_time          200000 non-null int64
title              200000 non-null object
type               200000 non-null object
world              200000 non-null object
dtypes: int64(3), object(8)
memory usage: 16.8+ MB


In [42]:
df_train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 11 columns):
installation_id    200000 non-null object
event_id           200000 non-null object
game_session       200000 non-null object
timestamp          200000 non-null object
event_data         200000 non-null object
event_count        200000 non-null int64
event_code         200000 non-null int64
game_time          200000 non-null int64
title              200000 non-null object
type               200000 non-null object
world              200000 non-null object
dtypes: int64(3), object(8)
memory usage: 147.4 MB


Обычный и дипмемори - как считается. Картинка

In [4]:
pandas_profiling.ProfileReport(df_train)

In [7]:
# json unpack
df_train_extracted = pd.io.json.json_normalize(df_train['event_data'].apply(json.loads))

Немного про json

In [13]:
pandas_profiling.ProfileReport(df_train_extracted, minimal=True)

TypeError: unhashable type: 'list'

Какие есть объекты в пандас и почему возникла проблема с хешем

In [10]:
df_train_extracted.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 136 columns):
event_code                  int64
event_count                 int64
version                     object
game_time                   float64
description                 object
identifier                  object
media_type                  object
total_duration              float64
duration                    float64
coordinates.x               float64
coordinates.y               float64
coordinates.stage_width     float64
coordinates.stage_height    float64
castles_placed              object
molds                       object
size                        float64
sand                        float64
filled                      object
level                       float64
round                       float64
movie_id                    object
options                     object
animals                     object
round_target.size           float64
round_target.type           object
rou

In [19]:
df_train_extracted[['flower', 'flowers']].dropna().head(10)

,flower,flowers
2022,0.0,"[0, 0, 0, 0, 0]"
2026,0.0,"[0, 0, 0, 0, 8]"
2032,5.0,"[8, 8, 8, 7, 8]"
2034,5.0,"[8, 8, 8, 7, 0]"
2035,1.0,"[8, 8, 8, 7, 0]"
2036,1.0,"[0, 8, 8, 7, 0]"
2037,2.0,"[0, 8, 8, 7, 0]"
2038,2.0,"[0, 0, 8, 7, 0]"
2040,3.0,"[0, 0, 8, 7, 0]"
2041,3.0,"[0, 0, 0, 7, 0]"


In [20]:
df_train_extracted['caterpillars'].dropna().head(10)

2685       [2, 2, 2]
2686      [11, 2, 2]
2687      [11, 2, 2]
2688     [11, 11, 2]
2689     [11, 11, 2]
2690    [11, 11, 11]
2692    [11, 11, 11]
2693    [11, 11, 11]
2694    [11, 11, 11]
2696    [11, 11, 11]
Name: caterpillars, dtype: object

In [40]:
df_train_extracted['bottles'].dropna().head(10)

3189    [{'color': 'blue', 'amount': 2}, {'color': 'blue', 'amount': 1}, {'color': 'purple', 'amount': 2...
3192    [{'color': 'blue', 'amount': 2}, {'color': 'blue', 'amount': 1}, {'color': 'purple', 'amount': 2...
3193    [{'color': 'blue', 'amount': 2}, {'color': 'blue', 'amount': 1}, {'color': 'purple', 'amount': 2...
3196    [{'color': 'blue', 'amount': 2}, {'color': 'blue', 'amount': 1}, {'color': 'purple', 'amount': 2...
3197    [{'color': 'blue', 'amount': 2}, {'color': 'blue', 'amount': 1}, {'color': 'purple', 'amount': 2...
3198    [{'color': 'blue', 'amount': 2}, {'color': 'blue', 'amount': 1}, {'color': 'purple', 'amount': 2...
3200    [{'color': 'blue', 'amount': 2}, {'color': 'blue', 'amount': 1}, {'color': 'purple', 'amount': 2...
3201    [{'color': 'blue', 'amount': 2}, {'color': 'blue', 'amount': 1}, {'color': 'purple', 'amount': 2...
3202    [{'color': 'blue', 'amount': 2}, {'color': 'blue', 'amount': 1}, {'color': 'purple', 'amount': 2...
3203    [{'color': 'blue', '

In [22]:
df_train_extracted.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 136 entries, event_code to chests
dtypes: float64(64), int64(2), object(70)
memory usage: 543.8 MB


Показть набор типичных проблем - пустые списки, булевые значения строками и т.д.. Дальше сносим лишние столбцы, что-бы не тратить время на распаковку

In [43]:
# reduce memory usage
df_train = reduce_mem_usage(df_train)

Mem. usage decreased to 13.73 Mb (18.2% reduction)


In [44]:
df_train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 11 columns):
installation_id    200000 non-null object
event_id           200000 non-null object
game_session       200000 non-null object
timestamp          200000 non-null object
event_data         200000 non-null object
event_count        200000 non-null int16
event_code         200000 non-null int16
game_time          200000 non-null int32
title              200000 non-null object
type               200000 non-null object
world              200000 non-null object
dtypes: int16(2), int32(1), object(8)
memory usage: 144.3 MB


Сравнение типов числе в пандас. Показать кусок кода

In [ ]:
# смотрим данные, опять что-то не так - недораспакованные строки, флоты, фокусы с типами (None и були), разреженность

In [ ]:
# объекты и категориальные типы - соращаем тут

In [ ]:
# замена пропусков

In [ ]:
# сериализация - нюансы

In [ ]:
# перегоняем в numpy и пример расчета модели, сравнение